<a id='top'></a>

# CSCI 3202, Spring 2020
# Assignment 3
# Due:  Wednesday 4 March 2020 by 11:59 PM

<br>

### Your name: Muntaha Pasha

<br>




**Some things that might be useful**


In [1]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time
import random

<br>

---

## Problem 1 (35 points):  Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>

<a id='p2a'></a>

### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **2b** and **2d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

**SOURCES USED:**

1 - (Get in Python) https://stackoverflow.com/questions/2068349/understanding-get-method-in-python

2 - (Class Definitions came from In-Class Notebook for Tic-Tac-Toe)

In [2]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = moves

class TicTacToe:
    def __init__(self, nrow=3, ncol=3, nwin=3, nexp=0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        self.nexp = nexp
        #Adding a value to keep track of expanded states.
        self.expandedStateVar = 0
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''        
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        return state.utility if player=='X' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    #self.display()
                    return self.state.utility                
def random_player(game):
    '''A player that chooses a legal move at random.'''
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]

<br>


### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [8]:
def random_player(arg1):
    actionsAvailable = arg1.state.moves
    moves = np.random.choice(len(actionsAvailable), 1)[0]
    randomMove = actionsAvailable[moves]
    return randomMove

#Winings
WinningGames = 0
#Losses
LosingGames = 0
#Ties
TiedGames = 0
#For 800 games...
for i in range(800):
    #Play Tic-Tac-Toe on a 3x3 board
    GamePlayed = TicTacToe(3,3,3)
    #Calculate the Score
    Score = GamePlayed.play_game(random_player, random_player)
    if Score == 1:
        WinningGames = WinningGames + 1
    elif Score == -1:
        LosingGames = LosingGames + 1
    elif Score == 0:
        TiedGames = TiedGames + 1
#Statistics of Game
WinTotals = 100*WinningGames/800
LostTotals = 100*LosingGames/800
TiedTotals = 100*TiedGames/800

print('Here are the Scores for Player 1')
print("------------------------------")
print("Wins: {}%".format(WinTotals))
print("Losses: {}%".format(LostTotals))
print("Ties: {}%".format(TiedTotals))

Here are the Scores for Player 1
------------------------------
Wins: 59.375%
Losses: 29.125%
Ties: 11.5%


**1B (Solution):**

So it seems like we have a higher winning streak! We win a lot more, lose a bit less, and tie with an even smaller probability!

<br>


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **2b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [9]:
#Winings
WinningGames = 0
#Losses
LosingGames = 0
#Ties
TiedGames = 0
#For 800 games...
for i in range(800):
    #Play Tic-Tac-Toe on a 4x4 board
    GamePlayed = TicTacToe(4,4,4)
    #Calculate the Score
    Score = GamePlayed.play_game(random_player, random_player)
    if Score == 1:
        WinningGames = WinningGames + 1
    elif Score == -1:
        LosingGames = LosingGames + 1
    elif Score == 0:
        TiedGames = TiedGames + 1
#Statistics of Game
WinTotals = 100*WinningGames/800
LostTotals = 100*LosingGames/800
TiedTotals = 100*TiedGames/800

print('Here are the Scores for Player 1')
print("------------------------------")
print("Wins: {}%".format(WinTotals))
print("Losses: {}%".format(LostTotals))
print("Ties: {}%".format(TiedTotals))

Here are the Scores for Player 1
------------------------------
Wins: 29.625%
Losses: 27.5%
Ties: 42.875%


**1C (Solution):**

Wow! Wayyy more Ties this time around! There is a very sharp decline in the Winning Streak we have, because our state space is larger on a $4$x$4$ board, so to achieve a win is going to be a lot harder on a larger board, especially when moves are all randomized too. It's not going to be easier to just win because the state space in itself is bigger. $\checkmark$

<br>


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

**SOURCES USED:**

1 - (Max/Mins) https://www.geeksforgeeks.org/max-min-python/

2 - (Infinity) https://intellipaat.com/community/28852/represent-infinity-as-an-integer-in-python-2-7

3 - (AlphaBeta Coding Example Chess) https://www.chessprogramming.org/Alpha-Beta

4 - (Minimax Code) https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-4-alpha-beta-pruning/

**CODE EXPLANATION:**

This was a hard algorithm to implement, but I managed to figure it out by looking at the examples above and researching more into what truly goes into the alphabeta algorithm. I started by defining my max_value and min_value functions inside of alphabeta search. The first thing I did was define alpha, and beta, then I got into my maximizer function.

**MAX:**

max_value takes alpha, beta, and the state. Game is passed through the parent function so we don't need to pass that into the parameters. The first thing I do is check if the game is over, and if so, return utlility, and None. Then I define some kind of ideal move that we have yet to discover, and my current value. In maximizer, that value is set to -infinity because we know everything will be better than that. I then look through my state moves, find the max between -infinity and whatever is returned from the minimizer (As the algorithm goes). Then I check if I've found a better value from the minimizer than -infinity. My pruning case comes in when we check the current value against beta, and if its bigger than, or equal to beta, we prune and return the new value without looking at the other branches. If so, I update my ideal move to the best maximum, and set alpha to the max between the alpha that was passed in, and the current value. 

**MIN:**

min_value takes alpha, beta, and the state. Game is passed through the parent function so we don't need to pass that into the parameters. The first thing I do is check if the game is over, and if so, return utlility, and None. Then I define some kind of ideal move that we have yet to discover, and my current value. In minimizer, that value is set to +infinity because we know everything will be lower than that. I then look through my state moves, find the min between +infinity and whatever is returned from the maximizer (As the algorithm goes). Then I check if I've found a lower value from the maximizer than +infinity. My pruning case comes in when we check the current value against alpha, and if its less than, or equal to alpha, we prune and return the new value without looking at the other branches. If so, I update my ideal move to the lowest minimum, and set beta to the min between the beta that was passed in, and the current value. 

Finally, at the end of the function, I check to see if the move was an "X" or an "O". X is a maximizer, so it calls max, and O is the minimizer, so it calls min. $\checkmark$

In [6]:
def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    beta = float('inf')
    alpha = -float('inf')
    #In-Function Functions that alpha-beta search uses.
    def max_value(state, alpha, beta):
        #If the game is over, then just return the state utility and none.
        if (game.game_over(state)):
            return (state.utility, None)
        idealMove = None
        current = -float('inf')
        #For each move in our state moves...
        for i in state.moves:
            checkerMove = max(current, min_value(game.result(i, state), alpha, beta)[0])
            #First we want to see if our current value is equal to the move, if not set it as that first.
            if current != checkerMove:
                current = checkerMove
                #Then set the ideal move to i, and keep going.
                idealMove = i
            #Now we check if the current value we have is going to be bigger than beta.
            #This is the pruning situation.
            if current >= beta:
                #If it is, set the ideal move to i again, and then return the current value and the ideal move.
                idealMove = i
                return (current, idealMove)
            #Now here is where we pick the maximum of either alpha, or current.
            alpha = max(current, alpha)
            #Update expanded states!
            game.expandedStateVar += 1
        #So return this.
        return (alpha, idealMove)
    def min_value(state, alpha, beta): 
        #If the game is over, then just return the state utility and none.
        if (game.game_over(state)):
            return (state.utility, None)
        idealMove = None
        current = float('inf')
        #For each move in our state moves...
        for i in state.moves:
            checkerMove = min(current, max_value(game.result(i, state), alpha, beta)[0])
            if current != checkerMove:
                current = checkerMove
                idealMove = i
            if current <= alpha:
                #If it is, set the ideal move to i again, and then return the current value and the ideal move.
                idealMove = i
                return (current, idealMove)
            #Now here is where we pick the maximum of either alpha, or current.
            beta = min(current, beta)
            #Update expanded states!
            game.expandedStateVar += 1
        #So return this.
        return (beta, idealMove)
    if game.state.to_move == "X":
        score = max_value
    #Otherwise, use the min value function.
    else:
        score = min_value
    #Finally, run and see what happens!
    x, y = score(game.state, alpha, beta)
    return y

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **only 10 games for each test**, and track the number of wins, draws and losses.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. 

**TEST CASE 1 - Alpha vs. Alpha**

Below is my testing for an alphabeta vs. alphabeta.

In [7]:
AA_win = 0
AA_lose = 0
AA_tie = 0
for i in range(0,10):
    gamePlayed = TicTacToe(3, 3, 3)
    score = gamePlayed.play_game(alphabeta_player, alphabeta_player)
    if score == 1:
        AA_win = AA_win + 1
    elif score == -1:
        AA_lose = AA_lose + 1
    else:
        AA_tie = AA_tie + 1 
        
AA_win_total = (AA_win/10)*100
AA_lose_total = (AA_lose/10)*100
AA_tie_total = (AA_tie/10)*100

print("Alphabeta vs. Alphabeta")
print("-------------------------")
print("Winnings: {}%".format(AA_win_total))
print("Losings: {}%".format(AA_lose_total))
print("Ties: {}%".format(AA_tie_total))
print(" ")

Alphabeta vs. Alphabeta
-------------------------
Winnings: 0.0%
Losings: 0.0%
Ties: 100.0%
 


**RESULTS**

"Two alpha-beta players should always draw, and should always end up in the same terminal state."

Yup! Ties is 100%. $\checkmark$

**TEST CASE 2 - Alpha vs. Random**

Below is my testing for an alphabeta vs. random player. $\checkmark$

In [67]:
AR_win = 0
AR_lose = 0
AR_tie = 0
for i in range(0,10):
    gamePlayed = TicTacToe(3, 3, 3)
    score = gamePlayed.play_game(alphabeta_player, random_player)
    if score == 1:
        AR_win = AR_win + 1
    elif score == -1:
        AR_lose = AR_lose + 1
    else:
        AR_tie = AR_tie + 1 
        
AR_win_total = (AR_win/10)*100
AR_lose_total = (AR_lose/10)*100
AR_tie_total = (AR_tie/10)*100

print("Alphabeta vs. Random Player")
print("-------------------------")
print("Winnings: {}%".format(AR_win_total))
print("Losings: {}%".format(AR_lose_total))
print("Ties: {}%".format(AR_tie_total))
print(" ")

Alphabeta vs. Random Player
-------------------------
Winnings: 100.0%
Losings: 0.0%
Ties: 0.0%
 


**RESULTS**

"An alpha-beta player who plays first should never lose to a random player who plays second."

Yup! Alphabeta wins 100% of the time. $\checkmark$

**TEST CASE 3 - Random vs. Alpha**

Below is my testing for a random player vs. alphabeta. $\checkmark$

In [74]:
RA_win = 0
RA_lose = 0
RA_tie = 0
for i in range(0,10):
    gamePlayed = TicTacToe(3, 3, 3)
    score = gamePlayed.play_game(random_player, alphabeta_player)
    if score == -1:
        RA_lose = RA_lose + 1
    elif score == 1:
        RA_win = RA_win + 1
    else:
        RA_tie = RA_tie + 1 
        
RA_win_total = (RA_win/10)*100
RA_lose_total = (RA_lose/10)*100
RA_tie_total = (RA_tie/10)*100

print("Random Player vs. Alphabeta")
print("-------------------------")
print("Winnings: {}%".format(RA_win_total))
print("Losings: {}%".format(RA_lose_total))
print("Ties: {}%".format(RA_tie_total))

Random Player vs. Alphabeta
-------------------------
Winnings: 0.0%
Losings: 100.0%
Ties: 0.0%


**RESULTS**

"A random player who plays first should never win to an alpha-beta player who plays second."

Yup! in my case above, when I executed it, Random Player loses 100% of the time. Sometimes its either a loss or a draw, (Like sometimes I get loss 70% and tie 30%, but never a win.) $\checkmark$

<br>


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, with and without pruning, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

**WITH PRUNING:**

Let's look at our algorithm from part 1d. We are pruning in this algorithm, so lets let it run and see how many nodes we expand, approximately.

In [52]:
def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    beta = float('inf')
    alpha = -float('inf')
    #In-Function Functions that alpha-beta search uses.
    def max_value(state, alpha, beta):
        #If the game is over, then just return the state utility and none.
        if (game.game_over(state)):
            return (state.utility, None)
        idealMove = None
        current = -float('inf')
        #For each move in our state moves...
        for i in state.moves:
            #Update expanded states!
            game.expandedStateVar += 1
            checkerMove = max(current, min_value(game.result(i, state), alpha, beta)[0])
            #First we want to see if our current value is equal to the move, if not set it as that first.
            if current != checkerMove:
                current = checkerMove
                #Then set the ideal move to i, and keep going.
                idealMove = i
            #Now we check if the current value we have is going to be bigger than beta.
            #This is the pruning situation.
            if current >= beta:
                #If it is, set the ideal move to i again, and then return the current value and the ideal move.
                idealMove = i
                return (current, idealMove)
            #Now here is where we pick the maximum of either alpha, or current.
            alpha = max(current, alpha)
        #So return this.
        return (alpha, idealMove)
    def min_value(state, alpha, beta): 
        #If the game is over, then just return the state utility and none.
        if (game.game_over(state)):
            return (state.utility, None)
        idealMove = None
        current = float('inf')
        #For each move in our state moves...
        for i in state.moves:
            #Update expanded states!
            game.expandedStateVar += 1
            checkerMove = min(current, max_value(game.result(i, state), alpha, beta)[0])
            if current != checkerMove:
                current = checkerMove
                idealMove = i
            if current <= alpha:
                #If it is, set the ideal move to i again, and then return the current value and the ideal move.
                idealMove = i
                return (current, idealMove)
            #Now here is where we pick the maximum of either alpha, or current.
            beta = min(current, beta)
        #So return this.
        return (beta, idealMove)
    if game.state.to_move == 'X':
        score = max_value
    #Otherwise, use the min value function.
    else:
        score = min_value
    #Finally, run and see what happens!
    x, y = score(game.state, alpha, beta)
    return y

**TESTING: PRUNING**

Let's test it.

In [56]:
#Let's do the pruning.
withPruning = TicTacToe(3,3,3)
#Call the function
alphabeta_player(withPruning)
#Print
print("Pruning expands approx. {} states.".format(withPruning.expandedStateVar))

Pruning expands approx. 18296 states.


**WITHOUT PRUNING:**

Let's look at our algorithm from part 1d. We are pruning in this algorithm, but if we get rid of the case of pruning entirely (everywhere we do a comparison between our current value and alpha or beta). So lets take that part of the code out, and let it run and see how many nodes we expand, approximately.

In [75]:
def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    beta = float('inf')
    alpha = -float('inf')
    #In-Function Functions that alpha-beta search uses.
    def max_value(state, alpha, beta):
        #If the game is over, then just return the state utility and none.
        if (game.game_over(state)):
            return (state.utility, None)
        idealMove = None
        current = -float('inf')
        #For each move in our state moves...
        for i in state.moves:
            #Update expanded states!
            game.expandedStateVar += 1
            checkerMove = max(current, min_value(game.result(i, state), alpha, beta)[0])
            #First we want to see if our current value is equal to the move, if not set it as that first.
            if current != checkerMove:
                current = checkerMove
                #Then set the ideal move to i, and keep going.
                idealMove = i
            #Now we check if the current value we have is going to be bigger than beta.
            #Now here is where we pick the maximum of either alpha, or current.
            alpha = max(current, alpha)
        #So return this.
        return (alpha, idealMove)
    def min_value(state, alpha, beta): 
        #If the game is over, then just return the state utility and none.
        if (game.game_over(state)):
            return (state.utility, None)
        idealMove = None
        current = float('inf')
        #For each move in our state moves...
        for i in state.moves:
            #Update expanded states!
            game.expandedStateVar += 1
            checkerMove = min(current, max_value(game.result(i, state), alpha, beta)[0])
            if current != checkerMove:
                current = checkerMove
                idealMove = i
            #Now here is where we pick the maximum of either alpha, or current.
            beta = min(current, beta)
        #So return this.
        return (beta, idealMove)
    if game.state.to_move == 'X':
        score = max_value
    #Otherwise, use the min value function.
    else:
        score = min_value
    #Finally, run and see what happens!
    x, y = score(game.state, alpha, beta)
    return y

**TESTING: WITHOUT PRUNING**

Let's test it.

In [76]:
#Let's do without pruning.
withoutPruning = TicTacToe(3,3,3)
#Call the function
alphabeta_player(withoutPruning)
#Print
print("Without Pruning expands approx. {} states.".format(withoutPruning.expandedStateVar))

Without Pruning expands approx. 549945 states.


**CONCLUSION:**

To reiterate, with pruning, we expanded 18,296 states, and without pruning, we expanded 549,945 states. That is a MASSIVE difference, and one that we expect because pruning cuts the amount of branches we need to explore when it finds the best option (lowest min possible or highest max possible). When you don't have pruning, you would need to explore states that won't make any change to the value the minimizer or maximizer has already chosen, so it seems futile to explore those. Hence pruning really does save the amount of work we have to do in this algorithm. $\checkmark$

<br>

---

## Problem 2 (30 points):  Uncertainty

### (2a) 

Suppose you are deciding when to arrive at a party. There is some optimal time to arrive when the loss you feel, as measured by _awkwardness_, is minimized at 0. That is, at some particular time, it is not awkward at all to show up to the party. The awkwardness (loss) increases as you arrive too early or too late relative to this optimal time. What is a suitable loss function, $L(d, x)$, to model this situation? Include definitions for $d$ and $x$, consistent with the examples from this class. Use this loss function this weekend when you go out.

**2a (Solution):**

$d$ is going to represent the loss, or in our case, the awkwardness. $x$ is going to represent the arrival time.

One way of doing this is making a Quadratic Function for Loss, where you have...

$$L(d,x) = (d-x)^2$$

This checks out! $\checkmark$

### (2b)

Suppose we have a situation where loss is given by the function $L(d, x) = 2(d-x)^2$. Set up, simplify, and evaluate integral(s) for the expected loss, $E_x[L(d, x)]$, where your prior beliefs regarding $x$ follow the distribution $f(x)$ given below. You may assume $f(x)=0$ for values of $x$ outside of the interval $[0, 3]$.

f(x) =  \begin{cases} 
      1/2 & 0 \leq x <1 \\
      3/8 & 1 \leq x <2 \\
      1/8 & 2 \leq x \leq 3 
   \end{cases}


$$\int_{0}^{1} 2(d-x)^2 \cdot \frac{1}{2} dx + \int_{1}^{2} 2(d-x)^2 \cdot \frac{3}{8} dx + \int_{2}^{3} 2(d-x)^2 \cdot \frac{1}{8} dx$$

$$-\frac{\left(d-1\right)^3}{3}+\frac{d^3}{3} + -\frac{\left(d-2\right)^3-\left(d-1\right)^3}{4} + -\frac{\left(d-3\right)^3-\left(d-2\right)^3}{12}$$

Get everything to the same denominator so you can add it. So this is...

$$(\frac{4}{4} \cdot -\frac{\left(d-1\right)^3}{3}) + (\frac{4}{4} \cdot \frac{d^3}{3}) + (\frac{3}{3} \cdot -\frac{\left(d-2\right)^3-\left(d-1\right)^3}{4}) + -\frac{\left(d-3\right)^3-\left(d-2\right)^3}{12}$$

Now everything has correct denominator.

$$-\frac{4\left(d-1\right)^3}{12}+\frac{4d^3}{12} -\frac{3\left(d-2\right)^3-\left(d-1\right)^3}{12} -\frac{\left(d-3\right)^3-\left(d-2\right)^3}{12}$$

Write it all over 12.

$$\frac{-4\left(d-1\right)^3 + 4d^3 - 3\left(d-2\right)^3-\left(d-1\right)^3 - \left(d-3\right)^3-\left(d-2\right)^3}{12}$$

Once that's expanded out we get...

$$-\frac{d^3}{2}+4d^2-\frac{15d}{2}+\frac{16}{3}$$

So that is going to be our expected loss, in terms of $d \checkmark$

### (2c) 

Suppose our expected loss is represented by the function $E_x[(L(d,x)]=(2-d)^2+2$, and our prior beliefs regarding $x$ are given by the distribution $f(x)$ from part b.

- Calculate Bayes' Decision, $d_{Bayes}$.

- Calculate the Expected Value of Including Uncertainty, EVIU. Suppose that if we ignore uncertainty, our best guess for what decision to make is the median of $x$ (under our prior $f(x)$).

**2c (Solution):**

$d_{bayes}$ is the minimum value of this function. The most minimum it can be is when $d$ is 2, because then you'll get...

$$(2-2)^2+2 = 0$$

It can not come lower than that. So $d_{bayes}=2 \checkmark$

Now for $EVIU$, we know the median is going to be $1$ based off our $f(x)$, and we know that the formula for $EVIU$ is $E_x[L(d_{iu},x)] - E_x[L(d_{Bayes},x)]$. Because our loss function is a quadratic, as discussed in the lecture, this implies symmetry, where $d_{iu} = d_{bayes} = E_x[x]$, so that means it's going to be $0$.

<br>

## Problem 3 (35 points): Maximizing some objective function with a Genetic Algorithm

Suppose we are trying to figure out a cookie recipe, but can't quite remember how much of each ingredient we need.

So we want to maximize the following objective function corresponding to how close we are to this recipe:

* 3/4 cup granulated sugar (36 tsp)
* 3/4 cup packed brown sugar  (36 tsp)
* 1 cup butter (48 tsp)
* 1 teaspoon vanilla (1 tsp)
* 1 egg
* 2 1/4 cups flour (108 tsp)
* 1 teaspoon baking soda (1 tsp)
* 1/2 teaspoon salt (0.5 tsp)
* 1 package (12 ounces) chocolate chips (2 cups) (96 tsp)

In [6]:
target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]

An example starting state for a member of our population might look like: $state = [30, 30, 40, 1, 0, 100, 0.5, 0.5, 100]$

### (3a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

**3A (Solution):**

I want to explain my code and how I wrote my fitness function. My fitness function takes some state of 9 ingredients, and compares those 9 ingredients to the 9 ingredients in our target recipe! So what this function returns is a fitness score of 9 elements that is the difference between target and state. Ideally what we want our genetic algorithm to do is to achieve a fitness score of 1000. Meaning there's no difference between the target and state values, but... we will have to see if that ends up happening. $\checkmark$

In [7]:
def recipe_success(state):
    #Let's start with a fitness of 100.
    fitnessScore = 100
    #For each element in the state...
    for i in range(len(state)):
        #For each state and target, find the difference between the two.
        subScore = state[i] - target[i]
        #If the difference is not 0, then find the fitnessScore by subtracting the absolute value of it.
        if subScore!=0:
            #Find the fitness score
            fitnessScore = fitnessScore - subScore
            #Take absolute value of it.
            fitnessScore = abs(fitnessScore)
        #If it is zero, then add 100.
        else:
            #So in the end we should have 1000 as our goal, if all elements match.
            fitnessScore = fitnessScore + 100
    #return the score of fitness.
    return fitnessScore

### (3b) 

Using our in class notebook "CSCI3202_GeneticAlgorithm.ipynb" as your guide, write a genetic algorithm that starts with a population of 10 randomly generated "recipes/states/members" and uses the objective function you wrote in **(3a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from numbers between 0 and 108 (half step intervals might be helpful since the recipe requires 1/2 tsp salt)
- Allow for mutations in your population with an overall probability of mutation set to p = 0.1
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 200 iterations ("generations"). Do you hit your target?

**SOURCES USED:**

1 - (arange) https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html

2 - (split) https://www.geeksforgeeks.org/python-custom-list-split/

In [30]:
#Reproduce
def reproduce(paren_1, paren_2):
    #   - 2 parents
    splittedValue = np.random.randint(low=1, high=len(paren_1))
    #   - Split parents 1 and 2
    firstHalf = paren_1[:splittedValue]
    #   - Find First Half (parent1) and Second Half (parent2)
    secondHalf = paren_2[splittedValue:]
    #   - Child is the First Half + Second Half
    childVal = firstHalf + secondHalf
    #Return Child
    return childVal

#Mutation
def mutate(childVal, sample_space):
    #   - Gene is the random int from 0 to child value
    geneValue = np.random.randint(low=0, high=len(childVal))
    #   - Mutate the child at the gene value chosen above from a member of sample space
    childVal[geneValue]=random.choice(sample_space)
    #Return Child
    return childVal

#Fitness
def fitness(population):
    #   - Reproduction Array
    p_reproduce = []
    #   - Fitness Array
    fitnessArr = []
    #   - For each i in the population
    for i in range(len(population)):
         #   - Performance is recipe_success of population at i.
        performanceVal = recipe_success(population[i])
         #   - Append performance to fitness array
        fitnessArr.append(performanceVal)
     #   - Sum up all the fitnesses
    totalSum = sum(fitnessArr)
     #   - For j in fitness array
    for j in fitnessArr:
        #Reproduce is gonna be the element of fitness over total sum
        reproduce = j/totalSum
        #Fitness Total is the appended reproduce
        fitnessTot = p_reproduce.append(reproduce)
    return fitnessTot

def genetic_algorithm(population, n_init, mutationProbability, populationSize, num_of_iterations, targetValue):
    samplePopulation = []
    #Population array
    poparr = []
    #Use arange to get step size of 0.5
    for member in np.arange(0, population, 0.5):
        #append the member to sample population
        samplePopulation.append(member)
    #for each val in population size
    for each in range(populationSize):
        #Initial pop arr
        initPop=[]
        #For each in n_init
        for k in range(n_init):
            #Pick random and append it to init pop
            rand = random.choice(samplePopulation)
            initPop.append(rand)
        poparr.append(initPop)
    #For i in iterations
    for i in range(num_of_iterations):
        #New generation declaration
        generation_new = []
        for j in range(populationSize):
            #Reproduction is the fitness score of pop arr
            reproduction = fitness(poparr)
            #Parent results is going to be random choice to pick parents
            parentResults = np.random.choice(range(0, populationSize), size=2, p=reproduction, replace=False)
            #Parents 1 and 2 will come from the pop array
            paren1,paren2 = poparr[parentResults[0]], poparr[parentResults[1]]
            #Child will be reproduced in function
            child = reproduce(paren1, paren2)
            #Mutation here to pick if the mutation takes place, its either true of false
            sub = 1-mutationProbability
            mutation = np.random.choice([True,False], p=[mutationProbability, sub])
            #If mutation is true
            if mutation:
                #Then child is the mutation
                child = mutate(child, samplePopulation)
            #Append the child to new generation
            generation_new.append(child)
        #Update population array
        poparr=generation_new
        #Get performance
        performance=[recipe_success(member) for member in poparr]
        best_member=max(zip(performance,poparr))
        if best_member[0]>=targetValue:
            return best_member
    print("reached n_iter")
    return False

In [36]:
print("Results are...")
print("-----------")
test1 = genetic_algorithm(population=108, n_init=9, mutationProbability=0.1, populationSize=10, num_of_iterations=200, targetValue=1000)
print(test1)

Results are...
-----------
reached n_iter
False


**3b (Solution):**

Nope, not hitting the target yet... 

### (3c)

Report the following:
- Report whether you minimized of maximized the objective function and whether that led to any major changes in how you designed the probability of reproduction. A couple sentences here is fine.

- Report how many ingredients you ended up matching. e.g. target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96], perhaps your algorithm achieved [36, 42, 8, 1, 1, 100, 56, 0.5, 0], then you would have matched 4 of the ingredient values.

- Report how many iterations you tried in order to get this answer. (Don't burn up your machine in the process)

**I'm going to re-implement my code with matching.**

In [56]:
# Your code here.
def reproduce(paren_1, paren_2):
    #   - 2 parents
    splittedValue = np.random.randint(low=1, high=len(paren_1))
    #   - Split parents 1 and 2
    firstHalf = paren_1[:splittedValue]
    #   - Find First Half (parent1) and Second Half (parent2)
    secondHalf = paren_2[splittedValue:]
    #   - Child is the First Half + Second Half
    childVal = firstHalf + secondHalf
    #Return Child
    return childVal

#Mutation
def mutate(childVal, sample_space):
    #   - Gene is the random int from 0 to child value
    geneValue = np.random.randint(low=0, high=len(childVal))
    #   - Mutate the child at the gene value chosen above from a member of sample space
    childVal[geneValue]=random.choice(sample_space)
    #Return Child
    return childVal

#Fitness
def fitness(population):
    #   - Reproduction Array
    p_reproduce = []
    #   - Fitness Array
    fitnessArr = []
    #   - For each i in the population
    for i in range(len(population)):
         #   - Performance is recipe_success of population at i.
        performanceVal = recipe_success(population[i])
         #   - Append performance to fitness array
        fitnessArr.append(performanceVal)
     #   - Sum up all the fitnesses
    totalSum = sum(fitnessArr)
     #   - For j in fitness array
    for j in fitnessArr:
        #Reproduce is gonna be the element of fitness over total sum
        reproduce = j/totalSum
        #Fitness Total is the appended reproduce
        fitnessTot = p_reproduce.append(reproduce)
    return fitnessTot

def genetic_algorithm(population, n_init, mutationProbability, populationSize, num_of_iterations, targetValue):
    samplePopulation = []
    #Population array
    poparr = []
    #Use arange to get step size of 0.5
    for member in np.arange(0, population, 0.5):
        #append the member to sample population
        samplePopulation.append(member)
    #for each val in population size
    for each in range(populationSize):
        #Initial pop arr
        initPop=[]
        #For each in n_init
        for k in range(n_init):
            #Pick random and append it to init pop
            rand = random.choice(samplePopulation)
            initPop.append(rand)
        poparr.append(initPop)
    #For i in iterations
    for i in range(num_of_iterations):
        #New generation declaration
        generation_new = []
        for j in range(populationSize):
            #Reproduction is the fitness score of pop arr
            reproduction = fitness(poparr)
            #Parent results is going to be random choice to pick parents
            parentResults = np.random.choice(range(0, populationSize), size=2, p=reproduction, replace=False)
            #Parents 1 and 2 will come from the pop array
            paren1,paren2 = poparr[parentResults[0]], poparr[parentResults[1]]
            #Child will be reproduced in function
            child = reproduce(paren1, paren2)
            #Mutation here to pick if the mutation takes place, its either true of false
            sub = 1-mutationProbability
            mutation = np.random.choice([True,False], p=[mutationProbability, sub])
            #If mutation is true
            if mutation:
                #Then child is the mutation
                child = mutate(child, samplePopulation)
            #Append the child to new generation
            generation_new.append(child)
        #Update population array
        poparr=generation_new
        #Get performance
        performance=[recipe_success(member) for member in poparr]
        best_member=max(zip(performance,poparr))
        if best_member[0]>=targetValue:
            return best_member
        target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]
        #Implementing Matching
        matchedVals = []
        for o in range(0,9):
            if target[o]==best_member[1][o]:
                matchedVals.append(best_member[1][o])
                print("match found! Ingredient {} with value {} was matched on iteration {}".format(o,matchedVals, i))
    print("reached n_iter")
    return False

test1 = genetic_algorithm(population=108, n_init=9, mutationProbability=0.1, populationSize=10, num_of_iterations=400, targetValue=1000)
print(test1)

match found! Ingredient 7 with value [0.5] was matched on iteration 3
match found! Ingredient 7 with value [0.5] was matched on iteration 4
match found! Ingredient 7 with value [0.5] was matched on iteration 5
match found! Ingredient 7 with value [0.5] was matched on iteration 6
match found! Ingredient 7 with value [0.5] was matched on iteration 7
match found! Ingredient 7 with value [0.5] was matched on iteration 8
match found! Ingredient 7 with value [0.5] was matched on iteration 9
match found! Ingredient 2 with value [48.0] was matched on iteration 195
match found! Ingredient 2 with value [48.0] was matched on iteration 196
match found! Ingredient 2 with value [48.0] was matched on iteration 197
match found! Ingredient 4 with value [1.0] was matched on iteration 276
reached n_iter
False


**3C (Solution):**

I maximized the objective function because I wanted the fitness score to be greater than its initial value by the time genetic algorithm has run. Ideally you'd want a score of 1000 at the end, (where the initial fitness score starts at 100 and adds 100 each time an element matches) but I haven't gotten close to that yet. It didn't really lead to major changing in reproduction, I just split the parents and made the child, regardless. $\checkmark$

In regards to matching, I've matched quite a few ingredients actually! Like when I ran it above, its matching different ingredients per different iteration! Like for instance, a sample result I got is.. 

**A sample output of my code:**

match found! Ingredient 7 with value [0.5] was matched on iteration 8

match found! Ingredient 7 with value [0.5] was matched on iteration 9

match found! Ingredient 2 with value [48.0] was matched on iteration 195

match found! Ingredient 2 with value [48.0] was matched on iteration 196

match found! Ingredient 2 with value [48.0] was matched on iteration 197

match found! Ingredient 4 with value [1.0] was matched on iteration 276


So I am definitely finding matches! Ingredients I'm matching on are 1 cup butter (48 tsp)
1 teaspoon vanilla (1 tsp), 1 egg, and a pkg of chocolate chips. Mostly I match on the ingredients with $1$, but I am matching other things from time to time as well.  I am doing it over $400$ iterations, but honestly even if I do about $200$, I still can find them. $\checkmark$